## Notebook for running estimates for simulated data 

This notebook is dedicated to a structured set of runs for 

(1) simulating a dynamic hierarchical DCSBM using make_hierarchical_model, 

(2) estimating super-/sub-community membership from the adjacency matrices using the temporally stable estimation algorithm, 

(3) measuring the performance of estimated membership vs ground truth, in terms of super and mean ARI 

Please refer to the report for details on the above.  

This code assumes a default set of parameters for make_hierarchical model, and 
for a set of parameters/tuples of parameters, an exploration list.  

For each exploration list, the model will cycle through the list, replacing in the 
current parameters the default value for the active parameter with the current list entry, 
simulating a model, estimating and measuring the ARIs.  This is done ten times per value, 
and mean and standard deviation are added to a dictionary.  The dictionary is saved after 
each list item is processed.  

This notebook runs for quite some time; on a Dell Precision 5490 it takes about 14 hours for a complete run.  
Reducing the number of runs from 10 to 2 helps of course, as does removing the very heavy calculations 
for n=2000 and T=50.  

The procedure is smart enough to recognize and continue results of partial runs, so one can stop and re-start 
computations.  It will for each inner loop over the 10 runs re-set the random seed to the value provided, hence 
the runs will still be reproducible.  


In [ ]:
# imports 
import numpy as np
import pandas as pd
import pickle
import os
#import copy
from copy import deepcopy as dc 

from datetime import datetime
from typing import Dict, List, Callable

from dyn_hdcsbm import make_hierarchical_model
from estimators_cleaned import (
    dynamic_hierarchical_dcsbm_detection_stable, 
    dynamic_hierarchical_dcsbm_detection_simple
)
from ari_test import evaluate_from_hcs


In [ ]:
# parameter ranges and default parameters 

default_params = {
    'n': 1000, 'T': 20, 
    'rho1': 0.05, 'rho2': 0.1, 'rho3': 0.4, 
    'stay_probs': [0.8, 0.8, 0.8], 
}
default_algo_params = {'stability_weight': 0} # TODO:  remove this from the logic 
par_ranges = {
    'varying_n': [200, 500, 1000, 1500, 2000],
    'varying_T': [5, 10, 20, 30, 50],
    'varying_rho_difference': [
        (0.05, 0.10, 0.15),  
        (0.05, 0.10, 0.25),  
        (0.05, 0.10, 0.40),  
        (0.05, 0.10, 0.60),  
        (0.05, 0.10, 0.90),  
        (0.05, 0.10, 0.95),  
    ],
    'varying_rho_magnitude': [
        (0.005, 0.010, 0.040), 
        (0.010, 0.020, 0.080),  
        (0.025, 0.050, 0.200),  
        (0.050, 0.100, 0.400),  
        (0.100, 0.200, 0.800),  
    ],
    'varying_stay_probs': [
        [0.2, 0.2, 0.2],  
        [0.4, 0.4, 0.4],  
        [0.6, 0.6, 0.6],  
        [0.8, 0.8, 0.8],  
        [0.9, 0.9, 0.9],  
        [0.95, 0.95, 0.95],  
    ]
}


In [ ]:
# help functions running experiments in loops  
class ExperimentManager:
    """Manages simulation/estimation experiment with automatic saving/loading.

    TODO:  Make more flexible by handing over estimator and parameter defaults/ranges.  
        
    """
    
    def __init__(self, output_dir: str = "output/stabilized", 
                 estimator: Callable = dynamic_hierarchical_dcsbm_detection_stable, 
                 random_seed=4321):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        self.estimator = estimator
        self.random_seed = random_seed

    def load_results(self, filename: str) -> Dict:
        """Load existing results from pickle file."""
        filepath = os.path.join(self.output_dir, filename)
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                data = pickle.load(f)
                print(f"Loaded existing results from {filename} with {len(data) - 1} entries")  # -1 for info
                return data
        else:
            print(f"Starting fresh results for {filename}")
            return {"info": {}}
    
    def save_results(self, filename: str, results: Dict) -> None:
        """Save results to pickle file."""
        with open(os.path.join(self.output_dir, filename), 'wb') as f:
            pickle.dump(results, f)
    
    def update_info(
            self, 
            results: Dict, 
            experiment_name: str, 
            defaults: Dict, 
            varying_param: str, 
            param_values: List
        ) -> None:
        """Update the info entry with experiment metadata.
        
        This is probably suboptimal, but for now it works.  
        """
        results["info"] = {
            "experiment_name": experiment_name,
            "default_parameters": defaults,
            "varying_parameter": varying_param,
            "parameter_values": param_values,
            "last_updated": datetime.now().isoformat(),
            "total_runs": len(param_values),
            "completed_runs": len(results) - 1  # -1 for info entry
        }
    
    def run_single_experiment(
            self, 
            params: Dict, 
            model_params: Dict
        ) -> Dict:
        """Run a single experiment with given parameters."""

        # run model and create adjacency matrices 
        model_params_copy = dc(model_params)
        model = make_hierarchical_model(**model_params_copy)
        
        adjacency_matrices = [
            model.generate_adjacency_matrix(t) 
            for t in range(model.generator.T + 1)
        ]
        
        # edge density for statistics 
        n = model_params['n']
        edge_density = np.mean([np.sum(A)/(n*(n-1)) for A in adjacency_matrices])
        
        # Estimation 
        algo_params_copy = dc(params)
        start_time = datetime.now()
        estimation_results = self.estimator(
            adjacency_matrices, 
            D=algo_params_copy.get('D', None), 
            verbose=False,
        )
        estimation_time = (datetime.now() - start_time).total_seconds()
        
        # Evaluate Estimation 
        results = evaluate_from_hcs(model, estimation_results)
        
        # Add metadata
        results['estimation_time'] = estimation_time
        results['edge_density'] = edge_density
        results['model_parameters'] = model_params
        #results['algorithm_parameters'] = params
        
        return results
    
    def print_results_summary(self, key: str, results: Dict):
        """Print summary of results for a single run."""
        print(f"\nResults for {key} :")
        print(f"  Super-community ARI: {results['super_ari']:.3f}")
        print(f"  Mean ARI: {results['mean_ari']:.3f}")
        print(f"  Edge density: {results.get('edge_density', 0):.4f}")
        print(f"  Estimation time: {results['estimation_time']:.1f} seconds")


def run_parameter_sweep(
    experiment_name: str,
    filename: str,
    parameter_generator: Callable,
    default_params: Dict,
    varying_param_name: str,
    manager: ExperimentManager,
    num_runs: int = 10  # Add parameter for number of runs
) -> Dict:
    """
    Run experiments over a list lof parameters with multiple runs per configuration.
    
    Parameters:
    -----------
    experiment_name: str
        Name for user display 
    filename: str
        where to write/read pickle file 
    parameter_generator : callable
        Generator that yields (key, model_params, algorithm_params) tuples
        This avoids repetitive code.  
    default_params : dict
        Default parameters for reference
    varying_param_name : str
        Name of the parameter being varied.  This is used for display and info.  
    manager : ExperimentManager
        Experiment manager instance
    num_runs : int
        Number of times to run each configuration (default: 10)
    """
    # Load existing results if existent  
    results = manager.load_results(filename)
    
    # Get all parameter configurations
    param_configs = list(parameter_generator())
    param_values = [config[0] for config in param_configs]
    
    # Update info
    manager.update_info(results, experiment_name, default_params, 
                       varying_param_name, param_values)
    
    print("*"*70)
    print(f"EXPERIMENT: {experiment_name}")
    print("*"*70)
    print(f"Default parameters: {default_params}")
    print(f"Varying: {varying_param_name}")
    print(f"Values: {param_values}")
    print(f"Runs per config: {num_runs}")
    print(f"Total configs: {len(param_configs)}")
    
    # Main loop to run experiments 
    for key, model_params, algo_params in param_configs:

        # Skip if already computed
        if key in results:
            print(f"Skipping {key} (already computed)")
            continue
        
        print(f"\n{'='*60}")
        print(f"Running experiment with {varying_param_name}={key}")
        print(f"{'='*60}")
        
        try:
            np.random.seed(manager.random_seed)

            # build a list of dicts, one per run 
            run_results = []
            for run_idx in range(num_runs):
                print(f"  Run {run_idx + 1}/{num_runs}...", end='', flush=True)
                
                
                # simulate and estimate.  Here we spend the majority of time.  
                # specifically for the simple estimator, we can run into errors, 
                # and specifically for this, we try each run 3 times if needed.  
                try:
                    model_params_copy, algo_params_copy = dc(model_params), dc(algo_params)
                    result = manager.run_single_experiment(
                        algo_params_copy, model_params_copy
                        )
                except Exception as e:
                    try: 
                        print("re-trying 1st time")
                        result = manager.run_single_experiment(
                            algo_params_copy, model_params_copy
                            )
                    except Exception as e:
                        print("re-trying 2nd time")
                        try: 
                            result = manager.run_single_experiment(
                                algo_params_copy, model_params_copy 
                                )
                        except Exception as e:
                            print(f"Error for {key}: {str(e)}")
                            raise e
                        
                run_results.append(result)                
                print(f" Super ARI: {result['super_ari']:.3f}, Mean ARI: {result['mean_ari']:.3f}")

            # collection of mean/std of relevant quantities             
            aggregated_result = {
                'super_ari': np.mean([r['super_ari'] for r in run_results]),
                'super_ari_std': np.std([r['super_ari'] for r in run_results]),
                'mean_ari': np.mean([r['mean_ari'] for r in run_results]),
                'mean_ari_std': np.std([r['mean_ari'] for r in run_results]),
                'estimation_time': np.mean([r['estimation_time'] for r in run_results]),
                'estimation_time_std': np.std([r['estimation_time'] for r in run_results]),
                'edge_density': np.mean([r.get('edge_density', 0) for r in run_results]),
                'model_parameters': model_params,
                'algorithm_parameters': algo_params,
                'num_runs': num_runs,
                'individual_runs': run_results  # Optional: keep individual results
            }
            
            # Store aggregated results for this parameter, and save as pickle file 
            results[key] = aggregated_result
            manager.save_results(filename, results)
            
            # Print summary
            print(f"\nAGGREGATED RESULTS for {key} (over {num_runs} runs):")
            print(f"  Super-community ARI: {aggregated_result['super_ari']:.3f} ± {aggregated_result['super_ari_std']:.3f}")
            print(f"  Time-average total ARI: {aggregated_result['mean_ari']:.3f} ± {aggregated_result['mean_ari_std']:.3f}")
            print(f"  Edge density: {aggregated_result.get('edge_density', 0):.4f}")
            print(f"  Estimation time: {aggregated_result['estimation_time']:.1f} ± {aggregated_result['estimation_time_std']:.1f} seconds")
            
        except Exception as e:
            print(f"Error for {key}: {str(e)}")
            raise e

    # Final save with updated info
    manager.update_info(
        results, experiment_name, default_params, 
        varying_param_name, param_values
        )
    manager.save_results(filename, results)
    
    print("\n" + "="*70)
    print(f"EXPERIMENT COMPLETE: {experiment_name}")
    print(f"Results saved to: {os.path.join(manager.output_dir, filename)}")
    print("="*70)
    
    return results

# Parameter generators for each experiment
# One can probably do this with a generator generating function, but this works.  
def generate_varying_n_params():

    for n in par_ranges['varying_n']:
        model_params = default_params.copy()
        model_params['n'] = n
        algo_params = {'stability_weight': 0}
        yield str(n), model_params, algo_params


def generate_varying_T_params():
    
    for T in par_ranges['varying_T']:
        model_params = default_params.copy()
        model_params['T'] = T
        algo_params = {'stability_weight': 0}
        yield str(T), model_params, algo_params


def generate_varying_rho_difference_params():
    
    for rho1, rho2, rho3 in par_ranges['varying_rho_difference']:
        model_params = default_params.copy()
        model_params.update({'rho1': rho1, 'rho2': rho2, 'rho3': rho3})
        algo_params = {'stability_weight': 0}
        yield f"({rho1},{rho2},{rho3})", model_params, algo_params


def generate_varying_rho_magnitude_params():

    for rho1, rho2, rho3 in par_ranges['varying_rho_magnitude']:
        model_params = default_params.copy()
        model_params.update({'rho1': rho1, 'rho2': rho2, 'rho3': rho3})
        algo_params = {'stability_weight': 0}
        yield f"({rho1},{rho2},{rho3})", model_params, algo_params


def generate_varying_stay_probs_params():

    for stay_probs in par_ranges['varying_stay_probs']:
        model_params = default_params.copy()
        model_params['stay_probs'] = stay_probs
        algo_params = {'stability_weight': 0}
        yield f"[{stay_probs[0]},{stay_probs[1]},{stay_probs[2]}]", model_params, algo_params


# Convenience functions for running specific experiments
def run_all_experiments(
        num_runs=10, 
        output_dir="output/tmp",
        estimator=dynamic_hierarchical_dcsbm_detection_stable, 
        random_seed=4321
        ):
    """Run all experiments with multiple runs per configuration.
    
    Sweep through every parameter in the par_ranges dictionary.  
    """
    manager = ExperimentManager(
        output_dir=output_dir, 
        estimator=estimator,
        random_seed=random_seed
        )
    
    # experiments contains name, filename, generator, param_name 
    experiments = [
        ("Varying Network Size (n)", "results_varying_n.pkl", generate_varying_n_params, "n"),
        ("Varying Time Points (T)", "results_varying_T.pkl", generate_varying_T_params, "T"),
        ("Varying Rho Difference", "results_varying_rho_difference.pkl", generate_varying_rho_difference_params, "rho_triplet"),
        ("Varying Rho Magnitude", "results_varying_rho_magnitude.pkl", generate_varying_rho_magnitude_params, "rho_triplet"),
        ("Varying Stay Probabilities", "results_varying_stay_probs.pkl", generate_varying_stay_probs_params, "stay_probs"),
    ]
    
    
    for exp_name, filename, generator, param_name in experiments:
        print(f"\n{'*='*40}")
        print(f"Starting: {exp_name}")
        print(f"{'*='*40}\n")
        
        run_parameter_sweep(
            experiment_name=exp_name, 
            filename=filename, 
            parameter_generator=generator, 
            default_params=default_params, 
            varying_param_name=param_name, 
            manager=manager, 
            num_runs=num_runs
            )
        
        print(f"\nCompleted: {exp_name}")
        print(f"{'='*80}\n")


def analyze_all_results(output_dir="output/tmp"):
    """Analyze results from all experiments."""
    manager = ExperimentManager(output_dir=output_dir)
    
    experiments = [
        ("Varying n", "results_varying_n.pkl"),
        ("Varying T", "results_varying_T.pkl"),
        ("Varying Rho Difference", "results_varying_rho_difference.pkl"),
        ("Varying Rho Magnitude", "results_varying_rho_magnitude.pkl"),
        ("Varying Stay Probabilities", "results_varying_stay_probs.pkl"),
    ]
    
    for exp_name, filename in experiments:
        print(f"\n{'+-'*40}")
        print(f"EXPERIMENT: {exp_name}")
        print(f"{'+-'*40}")
        
        results = manager.load_results(filename)
        
        if "info" in results:
            info = results["info"]
            print(f"Experiment details:")
            print(f"  Varying parameter: {info.get('varying_parameter', 'Unknown')}")
            print(f"  Completed runs: {info.get('completed_runs', 0)}/{info.get('total_runs', 0)}")
            print(f"  Last updated: {info.get('last_updated', 'Unknown')}")
            print()
        
        # Create summary table
        data = []
        for key, result in results.items():
            if key != "info" and 'error' not in result:
                # Check if this is a multi-run result
                if 'super_ari_std' in result:
                    data.append({
                        'Parameter': key,
                        'Runs': result.get('num_runs', 1),
                        'Super ARI': f"{result['super_ari']:.3f} ± {result['super_ari_std']:.3f}",
                        'Time-avg ARI': f"{result['mean_ari']:.3f} ± {result['mean_ari_std']:.3f}",
                        'Edge Density': f"{result.get('edge_density', 0):.4f}",
                        'Time (s)': f"{result['estimation_time']:.1f} ± {result['estimation_time_std']:.1f}"
                    })
                else:
                    # Legacy single-run format
                    data.append({
                        'Parameter': key,
                        'Runs': 1,
                        'Super ARI': f"{result['super_ari']:.3f}",
                        'Time-avg ARI': f"{result['mean_ari']:.3f}",
                        'Edge Density': f"{result.get('edge_density', 0):.4f}",
                        'Time (s)': f"{result['estimation_time']:.1f}"
                    })
        
        df = pd.DataFrame(data)
        print(df.to_string(index=False))


In [ ]:
# run all experiments for stabilized estimator 
seed = 4321 

run_all_experiments(
    num_runs=10, 
    output_dir="output/stabilized",
    estimator=dynamic_hierarchical_dcsbm_detection_stable,  
    random_seed=seed 
)
analyze_all_results(output_dir="output/stabilized")



In [ ]:
# run all experiments for simple estimator 

# NB, for this particular setting and n=500, one actually can run into errors 
# caused by degenerate estimates cases.  Specifically for this, I added 
# a loop trying three times if needed.  (This is not necessary for the stabilized estimator.)

par_ranges['varying_n'] = [500, 1000, 1500, 2000]

par_ranges['varying_rho_magnitude'] = [
        (0.010, 0.020, 0.080),  
        (0.025, 0.050, 0.200),  
        (0.050, 0.100, 0.400),  
        (0.100, 0.200, 0.800),  
    ]

run_all_experiments(
    num_runs=10, 
    output_dir="output/simple",
    estimator=dynamic_hierarchical_dcsbm_detection_simple, 
    random_seed=seed
)
analyze_all_results(output_dir="output/simple")